In [1]:
import os

In [4]:
%pwd

'/Users/shamalishinde/Akshay_Projects/Pycharm_Projects/text-summarizer-project'

In [3]:
os.chdir("../")

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [13]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config



In [15]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2024-12-25 20:17:38,758: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-25 20:17:38,761: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-25 20:17:38,762: INFO: common: created directory at: artifacts]
[2024-12-25 20:17:38,763: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-25 20:17:41,498: INFO: <ipython-input-16-6746d488a707>: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 4140833
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9f2051ee1574ca0ab38be0fba56cedea004c59a1c3060155a7a6dd24ec07702e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C325:1D2AD0:A2D0D:EEE7C:676C1B0B
Accept-Ranges: bytes
Date: Wed, 25 Dec 2024 14:47:41 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4